## LCEL (LangChain Expression Language)

1. 所有元件皆實作 Runnable 協定

| 方法        | 功能                   |
| --------- | -------------------- |
| `invoke`  | 同步呼叫                 |
| `ainvoke` | 非同步呼叫                |
| `stream`  | 串流輸出（token by token） |
| `astream` | 非同步串流輸出              |
| `batch`   | 一次處理多組輸入             |
| `abatch`  | 非同步 batch 處理         |

2. 通用屬性（Common Properties）：
- input_schema: 元件可以接受的輸入格式
- output_schema: 元件輸出的格式結構

3. 常見 LCEL 元件與 I/O 對照表

| Component      | Input Type                             | Output Type          |
| -------------- | -------------------------------------- | -------------------- |
| `Prompt`       | Dictionary                             | Prompt Value（LLM 可用） |
| `Retriever`    | Single String（如 query）                 | List of Documents    |
| `LLM`          | String, List of Messages, Prompt Value | String               |
| `ChatModel`    | String, List of Messages, Prompt Value | ChatMessage          |
| `Tool`         | String / Dictionary                    | 依 tool 定義而異          |
| `OutputParser` | LLM 或 ChatModel 的 Output               | 視 parser 而定          |


`Runnables` 支援
1. 
- Async：支援非同步 ainvoke() 執行，適用於等待外部 API 或 LLM 的情境。
- Batch：可一次處理多筆輸入（使用 batch() 或 abatch()）。
- Streaming：支援串流回應（例如 LLM token-by-token 的輸出）。

2. Fallbacks

提供「備援處理機制」，例如：

- 如果主模型失敗，使用備用模型。
- 如果某一步驟錯誤，退回預設輸出。
- 可以用在 `.with_fallbacks([r1, r2])` 等操作中

3. Parallelism（並行處理）

設計考量到 LLM 呼叫成本高、耗時。可以並行處理多個步驟：

- 如多個 Retriever、Tool、Chain。
- 透過 RunnableParallel, RunnableMap, RunnableSequence 等組件實現。

4. Logging
- 已內建追蹤與記錄功能（整合 langchain.debug, langsmith, logging）。
- 可用來檢查 Chain 執行細節、錯誤或效能瓶頸。

In [ ]:
from langchain_ollama.llms import OllamaLLM
chat = OllamaLLM(temperature=0.4, model="llama3")

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="summary", description="Summary of the input question"),
    ResponseSchema(name="category", description="One-word category of the question (e.g., math, general, science)")
]

# 2. 建立 JSON output parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [3]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You will be given a question. Your job is to return a JSON object with:
- a "summary" of the question
- a "category" for the question

Question: {question}

{format_instructions}
""").partial(format_instructions=parser.get_format_instructions())

print(prompt)

input_variables=['question'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"summary": string  // Summary of the input question\n\t"category": string  // One-word category of the question (e.g., math, general, science)\n}\n```'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='\nYou will be given a question. Your job is to return a JSON object with:\n- a "summary" of the question\n- a "category" for the question\n\nQuestion: {question}\n\n{format_instructions}\n'), additional_kwargs={})]


In [ ]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(
    prompt,
    chat,
    chat
)

response = chain.invoke({"question": "Why is the sky blue?"})
print(response)

{'summary': "The reason for the sky's blue color", 'category': 'science'}


In [6]:
sequence = prompt | chat | parser

sequence.invoke({"question": "What is 1+1"})

{'summary': 'Basic arithmetic', 'category': 'math'}

### More complex chain

In [1]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama

llm = ChatOllama(temperature=0.0, model="llama3.2")

embeddings = OllamaEmbeddings(model="llama3.2")

vectorstore = DocArrayInMemorySearch.from_texts(
    ["kube-apiserver: The core component server that exposes the Kubernetes HTTP API", "etcd: Consistent and highly-available key value store for all API server data"],
    embedding=embeddings
)
retriever = vectorstore.as_retriever()

/home/itachi/.local/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [2]:
retriever.invoke("What is kube-apiserver?")

[Document(metadata={}, page_content='kube-apiserver: The core component server that exposes the Kubernetes HTTP API'),
 Document(metadata={}, page_content='etcd: Consistent and highly-available key value store for all API server data')]

In [3]:
retriever.invoke("What is etcd?")

[Document(metadata={}, page_content='etcd: Consistent and highly-available key value store for all API server data'),
 Document(metadata={}, page_content='kube-apiserver: The core component server that exposes the Kubernetes HTTP API')]

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the question based only on the following context: {context}"),
        ("human", "{question}")
    ]
)

In [12]:
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

In [18]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | StrOutputParser()

In [19]:
chain

{
  context: RunnableLambda(...),
  question: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the question based only on the following context: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='llama3.2', temperature=0.0)
| StrOutputParser()

In [20]:
chain.invoke({"question": "What is kube-apiserver?"})

"Kube-apiserver is the core component of a Kubernetes cluster. It's responsible for exposing the Kubernetes REST API, which allows users to interact with the cluster and manage its resources, such as pods, services, and deployments. In other words, it acts as an entry point for the Kubernetes cluster, providing a interface for clients (such as the Kubernetes CLI or other applications) to access and manipulate cluster data."

## RunnableParallel

In [10]:
prompt_summary = ChatPromptTemplate.from_template("Summarize this: {text}")
prompt_translate = ChatPromptTemplate.from_template("Translate into Chinese Tranditional: {text}")

In [11]:
from langchain_core.output_parsers import StrOutputParser
summary_chain = prompt_summary | chat | StrOutputParser()
translate_chain = prompt_translate | chat | StrOutputParser()

In [12]:
from langchain_core.runnables import RunnableParallel
parallel_chain = RunnableParallel(
    summarize=summary_chain,
    translate=translate_chain
)

# 5. 執行
input_text = {"text": "LangChain is a powerful framework but has a steep learning curve."}
result = parallel_chain.invoke(input_text)

print("✅ Summary:", result["summarize"])
print("✅ Translate:", result["translate"])

✅ Summary: LangChain is a robust tool, but it requires a significant amount of effort and dedication to learn how to use it effectively.
✅ Translate: Here's the translation:

人工：LangChain 是 一個強大框架，但具有陡峭的學習曲線。

Traditional Chinese:



Note: "" means "steep" or "sharp", and "" means "learning curve". The phrase "" is a common idiomatic expression in Chinese that refers to something having a difficult learning process.


## RunnableBranch

In [25]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# Prompt: 一般問題
general_prompt = ChatPromptTemplate.from_template("Answer casually: {question}")
general_chain = general_prompt | chat | () | RunnableLambda(lambda output: {
        "source": "general",
        "result": output
    })
# Prompt: 數學問題
math_prompt = ChatPromptTemplate.from_template("You're a math tutor. Solve step by step: {question}")
math_chain = math_prompt | chat | StrOutputParser() | RunnableLambda(lambda output: {
    "source": "math",
    "result": output
})

In [26]:
from langchain_core.runnables import RunnableBranch
branch = RunnableBranch(
    (lambda x: "math" in x["question"].lower(), math_chain),
    general_chain  # 預設 fallback branch
)


In [27]:
# 測試一：一般問題
res1 = branch.invoke({"question": "Why is the sky blue?"})
print("☁️ General:", res1)


☁️ General: {'source': 'general', 'result': "You know, it's just one of those things that's always been there, right? But if you really want to get into it, I guess it has something to do with this thing called light scattering. See, when sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.\n\nSo, when we look up at the sky, our eyes see the combined effect of all that scattered blue light, making it appear blue to us! It's pretty cool, actually. And if you've ever seen a sunset or sunrise, you know that the color of the sky can change depending on the time of day and the amount of dust and water vapor in the air. But basically, the blue sky is just our atmosphere doing its thing!\n\nWhat do you think? Want to go stargazing tonight?"}


In [28]:

# 測試二：數學問題
res2 = branch.invoke({"question": "How to solve this math equation: 2x + 3 = 7?"})
print("📐 Math:", res2)

📐 Math: {'source': 'math', 'result': "I'd be happy to help you solve the equation 2x + 3 = 7.\n\nTo start, we can subtract 3 from both sides of the equation to get:\n\n2x = 7 - 3\n2x = 4\n\nNext, we can divide both sides of the equation by 2 to solve for x. This gives us:\n\nx = 4/2\nx = 2\n\nSo, the value of x is 2!"}
